# 1. Introduction
# FAISS, or Facebook AI Similarity Search,
is an open-source library that helps developers search for similar embeddings in multimedia documents:
## What it does
**FAISS** is a library that helps developers quickly search for **similar embeddings in multimedia documents.** It's designed to handle **large-scale datasets** and is a *key tool for applications in machine learning, artificial intelligence, and data science.*

## How it works
**FAISS** assumes that *instances are represented as vectors* and can be compared using **L2 (Euclidean) distances or dot products.** It uses a variety of algorithms and optimizations to ensure it remains at the forefront of vector database technology.

## Why it's useful
**FAISS** solves limitations of traditional query search engines, which are optimized for hash-based searches. It's a valuable tool for applications that require rapid and accurate similarity searches.

## Some of its features
FAISS includes a variety of index structures, including:

* **IndexIVFFlat:** Uses an inverted file system to divide the dataset into clusters and assign a list of vectors to each cluster. This index structure is suitable for large-scale applications.
* **IndexIVFPQ:** Combines Product Quantization (PQ) and an inverted file system to store and retrieve high-dimensional embeddings.

## Where to learn more
* You can learn more about FAISS from the [Faiss documentation](https://faiss.ai/?form=MG0AV3), [GitHub](https://github.com/facebookresearch/faiss?form=MG0AV3), and other resources.
* [Implementing FAISS: Vector Similarity Search for Recommendations](https://manangarg.medium.com/implementing-faiss-vector-similarity-search-for-recommendations-faa5149f55de)

# 2. Install libraries

In [ ]:
# Installing relevant libraries
!pip install sentence_transformers
!pip install datasets
!pip install faiss-gpu
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 75.3 MB/s eta 0:00:00


# 3. Import libraries

In [ ]:
from datasets import load_dataset
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import warnings

# 4. Set Options
* *ignore Warning*
* *floating number display options*

In [ ]:
warnings.simplefilter('ignore')
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:.2f}'.format

# 5. Load dataset
Let's use a small dataset of book titles and their descriptions for our use-case.

In [ ]:
dataset = load_dataset('Skelebor/book_titles_and_descriptions_en_clean', split='test')
df = pd.DataFrame(dataset)
df.head()

dataset_infos.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

(…)-00000-of-00002-7ed8cdce71e9d933.parquet:   0%|          | 0.00/306M [00:00<?, ?B/s]

(…)-00001-of-00002-68a449783d5db899.parquet:   0%|          | 0.00/306M [00:00<?, ?B/s]

(…)-00000-of-00001-0ce6014f3ee7e1e3.parquet:   0%|          | 0.00/34.0M [00:00<?, ?B/s]

(…)-00000-of-00001-b285c92e4abb7e76.parquet:   0%|          | 0.00/33.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1032335 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/57352 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/57352 [00:00<?, ? examples/s]

,title,description
0,The Baby of Their Dreams,"Barcelona, baby...bride?\nSeven years ago ER d..."
1,"Air Gear, Vol. 8 (Air Gear, #8)",Behemoth has already taken out part of Ikki's ...
2,Walking Over Eggshells,Walking Over Eggshells is an autobiography tha...
3,"Charmed (Fairy Tale Reform School, #2)",Charmed is the exciting sequel to the wildly p...
4,"Blown Away (Unconventional in Atlanta, #2)",Sometimes love finds you before you think you'...


# 6. Dataset EDA
* shape
* redundant data
* null

In [ ]:
# Shape of the dataset -
print("Shape of dataset".ljust(50, '.'), df.shape)

# Removing duplicate rows
df.drop_duplicates(inplace=True)

# Shape of the dataset after removing duplicates -
print("Shape of dataset after removing duplicates".ljust(50, '.'), df.shape)

# Checking number of Nulls
df.isnull().sum(axis=1).sum()
df.isnull().sum()

Shape of dataset.................................. (57352, 2)
Shape of dataset after removing duplicates........ (56889, 2)


,0
title,0
description,0


# 7. Embeddings
We’ll gather all the **descriptions** and create **sentence** embeddings for each one, which can subsequently be stored in FAISS.

In [ ]:
# creating list of sentences
sentences = df.description.to_list()
sentences

["Barcelona, baby...bride?\nSeven years ago ER doctor Cat Hayes was left heartbroken after losing her baby boy. Now she's focused on her career. But when she meets gorgeous Dr. Dominic Edwards at a Spanish conference, resisting his scorching touch isn't easy... Cat returns home sun-kissed and accidentally pregnant! \nWidower Dom never thought he'd ever find love again--let alone a family! As the promise of their miracle baby begins to heal boththeir hearts, Dom knows he can't let Cat slip through his fingers. All it takes is one down-on-one-knee question...",
 "Behemoth has already taken out part of Ikki's Air Trek team. Now Ikki and Onigiri are faced with their toughest opponents yet: a deadly femme fatale known as the Gorgon and the unstoppable punching machine, Bandou. Ikki's team has come a long way in their battle for Air Trek supremacy. Can they defeat Behemoth and get back in the game?",
 'Walking Over Eggshells is an autobiography that tells the story of a mentally abused child

constructing the sentence embeddings.

In [ ]:
# Creating sentence embeddings
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentence_embeddings = model.encode(sentences)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# 8. Vector Similarity Search Implementation
We configure the dimensionality of our FAISS database according to these vector embeddings.

In [ ]:
# Vector dimensionality
vec_dim = sentence_embeddings.shape[1]
vec_dim

768

In [ ]:
# Setting number of nearest neighbours
k = 4
# Creating Search Query embedding
query = model.encode(["Aliens and Spaceships"])

# 9.1 Flat L2 Index
We begin by initializing the flat **L2** distance index **‘IndexFlatL2’**, where we only need to specify the vector dimensionality. In this case, the dimensionality is **vec_dim = 768,** which corresponds to the output embeddings size of the **sentence-BERT** model.

In [ ]:
# Creating index
index = faiss.IndexFlatL2(vec_dim)
index.add(sentence_embeddings)

Once we are satisfied that our index is prepared, we proceed to add new vectors using the add method.

Next, we’ll locate the nearest query embeddings to our search query embedding based on the specified value of ‘k’.

In [ ]:
# Searching neighbours for the query
%%time
D, I = index.search(query, k)
print(I)

[[21656 44760 10863 35757]]
CPU times: user 17.5 ms, sys: 0 ns, total: 17.5 ms
Wall time: 16.5 ms


After selecting the descriptions that closely match the search query, we print the respective titles of the books associated with those descriptions.

In [ ]:
# Printing nearest neighbours for book titles
[f'{i}: {df.iloc[i].title}' for i in I[0]]

['21656: Aliens; Extraterresterial Tales of Terror',
 '44760: Edward Built a Rocketship',
 '10863: Across the Sea of Suns (Galactic Center, #2)',
 '35757: Look Inside Space']

# 9.2 Adding Partitioning to the Index
**FAISS** indeed provides the option to enhance our search efficiency through various methods, with partitioning the index into [Voronoi cells](https://en.wikipedia.org/wiki/Voronoi_diagram) being a popular approach. With this method, we take our query vector, determine the cell it belongs to, and then utilize our ‘IndexFlatL2’ to search among the query vector and all indexed vectors within that cell. Additionally, we can include vectors from other nearby cells if needed.

We can also adjust the number of nearby cells to search using the parameter ‘nprobe’.

In [ ]:
# Defining number of partitions
nlist = 200

In this updated setup, we’ve introduced a new parameter called ‘nlist’. This parameter allows us to specify the number of partitions we want our index to have.

In [ ]:
# Creating index
quantizer = faiss.IndexFlatL2(vec_dim)
index = faiss.IndexIVFFlat(quantizer, vec_dim, nlist)

We initialize our new partitioned index by incorporating our previous ‘IndexFlatL2’ operation as a quantization step, which serves as another stage in the search process. Then, we feed this into the new ‘IndexIVFFlat’ operation.

In [ ]:
# Training index
index.train(sentence_embeddings)

In [ ]:
# Increasing number of nearby cells to search
index.nprobe = 20

In [ ]:
# Searching neighbours for the query
%%time
D, I = index.search(query, k)
print(I)

[[-1 -1 -1 -1]]
CPU times: user 1.81 ms, sys: 14 µs, total: 1.82 ms
Wall time: 3.02 ms


In [ ]:
# Printing nearest neighbours for book titles
[f'{i}: {df.iloc[i].title}' for i in I[0]]

['-1: The Mortal Heart (Beautiful Creatures: The Untold Stories, #1)',
 '-1: The Mortal Heart (Beautiful Creatures: The Untold Stories, #1)',
 '-1: The Mortal Heart (Beautiful Creatures: The Untold Stories, #1)',
 '-1: The Mortal Heart (Beautiful Creatures: The Untold Stories, #1)']

# 9.3 Quantization
Now that we have significantly reduced the search time, the next step is to address the issue of storage space consumption when storing full vectors, especially in large datasets.

Fortunately, FAISS offers a solution by providing the ability to compress vectors using transformations based on **Product Quantization (PQ)**. But what exactly is PQ?

PQ can be seen as an additional approximation step, akin to our use of **‘IVF’**, which allowed us to approximate by reducing the scope of our search. However, PQ differs slightly as it approximates the distance (or similarity) calculation instead.

PQ achieves this by compressing the vectors themselves, which involves several steps:

1. We partition each original vector into several subvectors.
2. For each set of subvectors, we conduct a clustering operation, generating numerous centroids for each set of subvectors.
3. Within our vector of subvectors, we replace each subvector with the ID of its nearest centroid.
This compression process helps alleviate the storage space issue associated with storing full vectors.

In [ ]:
# Number of centroid IDs
num_cent_ids = 8

# Number of bits in each centroid
cent_bits = 8

In [ ]:
# Keeping the same L2 distance flat index
quantizer = faiss.IndexFlatL2(vec_dim)
index = faiss.IndexIVFPQ(quantizer, vec_dim, nlist, num_cent_ids, cent_bits)

Once more, we’ll have to train the index and add our vectors before proceeding further.

In [ ]:
# Training index
index.train(sentence_embeddings)

# Adding sentence embeddings
index.add(sentence_embeddings)

Let’s compare it with our previous index, which didn’t utilize Product Quantization (PQ) and had a ‘nprobe’ value of 20.

In [ ]:
# Defining number of nearby cells to search
index.nprobe = 20

In [ ]:
# Searching neighbours for the query
%%time
D, I = index.search(query, k)
print(I)

[[21656 28903 42857 25568]]
CPU times: user 1.63 ms, sys: 0 ns, total: 1.63 ms
Wall time: 1.64 ms


By incorporating **Product Quantization (PQ)**, we have successfully reduced our search time. Although the difference may be small on a dataset of this size, it can make a significant impact when scaled to larger datasets.

It’s important to note that with these speed optimization techniques (IVF and PQ), there might be slight differences in the results returned. However, upon printing out these results, we’ll observe that each item is still a relevant match.